In [3]:
%load_ext autoreload
%autoreload 2

import crud as crud
import spotify

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
sp = spotify.load_and_auth_spotify('../credentials.json')

In [ ]:
conn = crud.create_connection("../db/track_metadata.db")
tables = crud.get_tables(conn)

for table_name in tables.name.tolist():
    print(table_name)
    records = crud.get_records(conn, table_name)
    record_features = ['track_id', 'title', 'song_id', 'release', 
                       'artist_id', 'artist_mbid', 'artist_name', 
                       'year', 'artist_familiarity', 'artist_hotttnesss',
                       'query'
                      ]
uri = [spotify.get_spotify_uri(sp, row) for index, row in records.iterrows()]

songs


In [ ]:
len(uri)

In [11]:
records.head()

,track_id,title,song_id,release,artist_id,artist_mbid,artist_name,duration,artist_familiarity,artist_hotttnesss,year,track_7digitalid,shs_perf,shs_work,query
0,TRMMMYQ128F932D901,Silent Night,SOQMMHC12AB0180CB8,Monster Ballads X-Mas,ARYZTJS1187B98C555,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,252.05506,0.649822,0.394032,2003,7032331,-1,0,Silent Night Monster Ballads X-Mas
1,TRMMMKD128F425225D,Tanssi vaan,SOVFVAK12A8C1350D9,Karkuteillä,ARMVN3U1187FB3A1EB,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,156.55138,0.439604,0.356992,1995,1514808,-1,0,Tanssi vaan Karkuteillä
2,TRMMMRX128F93187D9,No One Could Ever,SOGTUKN12AB017F4F1,Butter,ARGEKB01187FB50750,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,138.97098,0.643681,0.437504,2006,6945353,-1,0,No One Could Ever Butter
3,TRMMMCH128F425532C,Si Vos Querés,SOBNYVR12A8C13558C,De Culo,ARNWYLR1187B9B2F9C,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,145.05751,0.448501,0.372349,2003,2168257,-1,0,Si Vos Querés De Culo
4,TRMMMWA128F426B589,Tangle Of Aspens,SOHSBXH12A8C13B0DF,Rene Ablaze Presents Winter Sessions,AREQDTE1269FB37231,,Der Mystic,514.29832,0.000000,0.000000,0,2264873,-1,0,Tangle Of Aspens Rene Ablaze Presents Winter S...


In [27]:
spotify.get_audio_analysis(sp, uri[0]['uri'])

{'uri': None,
 'duration': 253.02667,
 'loudness': -4.893,
 'tempo': 174.04,
 'tempo_confidence': 0.6,
 'time_signature': 4,
 'time_signature_confidence': 1.0,
 'key': 0,
 'key_confidence': 1.0,
 'mode': 1,
 'mode_confidence': 0.783}

In [51]:
spotify.get_audio_features([uri[0]['uri'], uri[1]['uri']])

[{'danceability': 0.453, 'energy': 0.915, 'key': 0, 'loudness': -4.893, 'mode': 1, 'speechiness': 0.0354, 'acousticness': 5.13e-06, 'instrumentalness': 0.631, 'liveness': 0.119, 'valence': 0.66, 'tempo': 174.04, 'type': 'audio_features', 'id': '3ZqUZpeWJ4norA5nvrE98c', 'uri': 'spotify:track:3ZqUZpeWJ4norA5nvrE98c', 'track_href': 'https://api.spotify.com/v1/tracks/3ZqUZpeWJ4norA5nvrE98c', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3ZqUZpeWJ4norA5nvrE98c', 'duration_ms': 253027, 'time_signature': 4}]


[{'uri': 'spotify:track:3ZqUZpeWJ4norA5nvrE98c',
  'danceability': 0.453,
  'energy': 0.915,
  'key': 0,
  'loudness': -4.893,
  'mode': 1,
  'speechiness': 0.0354,
  'acousticness': 5.13e-06,
  'instrumentalness': 0.631,
  'liveness': 0.119,
  'valence': 0.66,
  'tempo': 174.04}]

In [7]:
# query string should be song title + song artist
results = sp.search(q='Silent Night Monster Ballads X-Mas', limit=1)
for idx, track in enumerate(results['tracks']['items']):
    print(idx, track['name'], track['uri'])
    
    # TODO: add track['uri'] to list to run below jobs as batch. unclear about upper limit of each batch size
    analysis = sp.audio_analysis(track['uri']) # this can be batched to reduce api hits
    audio_analysis_features = ['uri', 'duration', 'loudness', 'tempo', 'tempo_confidence',
                               'time_signature', 'time_signature_confidence', 'key',
                                'key_confidence', 'mode', 'mode_confidence'
                              ]
    print({feature: analysis['track'].get(feature) for feature in audio_analysis_features})
    
    audio_features = sp.audio_features(track['uri']) # this can be batched to reduce api hits
    audio_features_features = ['uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 
                               'speechiness', 'acousticness', 'instrumentalness', 
                               'liveness', 'valence', 'tempo'
                              ]
    print(audio_features)

0 Silent Night spotify:track:3ZqUZpeWJ4norA5nvrE98c
{'duration': 253.02667, 'loudness': -4.893, 'tempo': 174.04, 'tempo_confidence': 0.6, 'time_signature': 4, 'time_signature_confidence': 1.0, 'key': 0, 'key_confidence': 1.0, 'mode': 1, 'mode_confidence': 0.783}
[{'danceability': 0.453, 'energy': 0.915, 'key': 0, 'loudness': -4.893, 'mode': 1, 'speechiness': 0.0354, 'acousticness': 5.13e-06, 'instrumentalness': 0.631, 'liveness': 0.119, 'valence': 0.66, 'tempo': 174.04, 'type': 'audio_features', 'id': '3ZqUZpeWJ4norA5nvrE98c', 'uri': 'spotify:track:3ZqUZpeWJ4norA5nvrE98c', 'track_href': 'https://api.spotify.com/v1/tracks/3ZqUZpeWJ4norA5nvrE98c', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3ZqUZpeWJ4norA5nvrE98c', 'duration_ms': 253027, 'time_signature': 4}]
